# Tool for Analysis of Stock Market Assets.

# In this notebook, we analyze stock market data collected via the Polygon.IO API.

## Flow:

1. &ensp;Data Downloading
2. &ensp;Reading & Understanding Data Types
3. &ensp;Data Preprocessing (Variable Selection)
4. &ensp;Data Exploration&ensp;
    1. &ensp;Exploring Historical Stock Trends (--using interactive candlestick charts--)
    2. &ensp;Exploring Stock Buy/Sell Strategies (--by analysis of EMS,&ensp;SMA&ensp;and RSI Indicators, and corresponding interactive charts--)

The Five Stock Market Assets Under Consideration in this Project are: 
1. &ensp;APPLE INC **(AAPL)**,
2. &ensp;TESLA INC **(TSLA)**, 
3. &ensp;META PLATFORMS INC **(META)**, 
4. &ensp;ALPHABET INC CLASS C **(GOOG)**, 
5. &ensp;AMAZON.COM INC **(AMZN)**

## Import packages:

In [1]:
#imports for data collection via PolygonAPI
from polygon import rest
from typing import cast
from urllib3 import HTTPResponse
import config #where an individual user's API_Key should be stored

#imports for data processing and visualization
import json
import pandas as pd
import numpy as np
import datetime as dt
from dash import Dash, html, dcc
from dash.dependencies import Input, Output
import matplotlib
matplotlib.use('Agg')
from plotly import graph_objects as go

#import for data backtesting and technical analysis
import talib
from backtesting import Backtest, Strategy
from backtesting.lib import crossover
from backtesting.test import SMA
import warnings
warnings.filterwarnings('ignore')

C:\Users\xzl5\AppData\Local\miniconda3\envs\drexel-project\Lib\site-packages\backtesting\_plotting.py:50: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support (e.g. PyCharm, Spyder IDE). Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '


Loading BokehJS ...

C:\Users\xzl5\AppData\Local\miniconda3\envs\drexel-project\Lib\site-packages\backtesting\test\__init__.py:8: FutureWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  return pd.read_csv(join(dirname(__file__), filename),
C:\Users\xzl5\AppData\Local\miniconda3\envs\drexel-project\Lib\site-packages\backtesting\test\__init__.py:8: FutureWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  return pd.read_csv(join(dirname(__file__), filename),


# 1. Data Downloading 

In [2]:
client = rest.RESTClient(config.API_KEY)

aggs = cast(
    HTTPResponse,
    client.get_aggs(
        'AAPL',
        1,
        'day',
        '2014-09-29',
        '2023-09-29',
        adjusted=True,
        raw=True,
        limit=50000
    )
)

poly_data = json.loads(aggs.data)


## Raw data (JSON standarized format)

In [3]:
print(poly_data)

{'ticker': 'AAPL', 'queryCount': 2267, 'resultsCount': 2267, 'adjusted': True, 'results': [{'v': 199064828.0, 'vw': 24.9605, 'o': 24.6625, 'c': 25.0275, 'h': 25.1088, 'l': 24.6575, 't': 1411963200000, 'n': 220213}, {'v': 221050556.0, 'vw': 25.2512, 'o': 25.2025, 'c': 25.1875, 'h': 25.385, 'l': 25.1325, 't': 1412049600000, 'n': 234940}, {'v': 205963876.0, 'vw': 24.8865, 'o': 25.1475, 'c': 24.795, 'h': 25.1725, 'l': 24.675, 't': 1412136000000, 'n': 251176}, {'v': 191027712.0, 'vw': 24.8188, 'o': 24.8175, 'c': 24.975, 'h': 25.055, 'l': 24.51, 't': 1412222400000, 'n': 225041}, {'v': 173878340.0, 'vw': 24.9318, 'o': 24.86, 'c': 24.905, 'h': 25.0525, 'l': 24.76, 't': 1412308800000, 'n': 211827}, {'v': 148204728.0, 'vw': 24.9937, 'o': 24.9875, 'c': 24.905, 'h': 25.1625, 'l': 24.855, 't': 1412568000000, 'n': 186768}, {'v': 168376676.0, 'vw': 24.8676, 'o': 24.8575, 'c': 24.6875, 'h': 25.03, 'l': 24.6825, 't': 1412654400000, 'n': 204987}, {'v': 229616496.0, 'vw': 24.9642, 'o': 24.69, 'c': 25.2, 

In [38]:
# save_file = open("META.json", "w")  
# json.dump(poly_data, save_file)  
# save_file.close() 

# 2. Data Reading & Understanding

## Reading data

In [4]:
df = pd.DataFrame(poly_data)
with pd.option_context('display.max_rows', 10, 'display.max_columns', None,'display.max_colwidth', None):
    display(df)

,ticker,queryCount,resultsCount,adjusted,results,status,request_id,count
0,AAPL,2267,2267,True,"{'v': 199064828.0, 'vw': 24.9605, 'o': 24.6625, 'c': 25.0275, 'h': 25.1088, 'l': 24.6575, 't': 1411963200000, 'n': 220213}",OK,44d919c8772464f0139c6579bd278533,2267
1,AAPL,2267,2267,True,"{'v': 221050556.0, 'vw': 25.2512, 'o': 25.2025, 'c': 25.1875, 'h': 25.385, 'l': 25.1325, 't': 1412049600000, 'n': 234940}",OK,44d919c8772464f0139c6579bd278533,2267
2,AAPL,2267,2267,True,"{'v': 205963876.0, 'vw': 24.8865, 'o': 25.1475, 'c': 24.795, 'h': 25.1725, 'l': 24.675, 't': 1412136000000, 'n': 251176}",OK,44d919c8772464f0139c6579bd278533,2267
3,AAPL,2267,2267,True,"{'v': 191027712.0, 'vw': 24.8188, 'o': 24.8175, 'c': 24.975, 'h': 25.055, 'l': 24.51, 't': 1412222400000, 'n': 225041}",OK,44d919c8772464f0139c6579bd278533,2267
4,AAPL,2267,2267,True,"{'v': 173878340.0, 'vw': 24.9318, 'o': 24.86, 'c': 24.905, 'h': 25.0525, 'l': 24.76, 't': 1412308800000, 'n': 211827}",OK,44d919c8772464f0139c6579bd278533,2267
...,...,...,...,...,...,...,...,...
2262,AAPL,2267,2267,True,"{'v': 46172740.0, 'vw': 175.6665, 'o': 174.2, 'c': 176.08, 'h': 176.97, 'l': 174.15, 't': 1695614400000, 'n': 564218}",OK,44d919c8772464f0139c6579bd278533,2267
2263,AAPL,2267,2267,True,"{'v': 64548945.0, 'vw': 172.7156, 'o': 174.82, 'c': 171.96, 'h': 175.2, 'l': 171.66, 't': 1695700800000, 'n': 688340}",OK,44d919c8772464f0139c6579bd278533,2267
2264,AAPL,2267,2267,True,"{'v': 66920708.0, 'vw': 170.5335, 'o': 172.62, 'c': 170.43, 'h': 173.04, 'l': 169.05, 't': 1695787200000, 'n': 707850}",OK,44d919c8772464f0139c6579bd278533,2267
2265,AAPL,2267,2267,True,"{'v': 56294419.0, 'vw': 170.297, 'o': 169.34, 'c': 170.69, 'h': 172.03, 'l': 167.62, 't': 1695873600000, 'n': 618584}",OK,44d919c8772464f0139c6579bd278533,2267


## Understanding data

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2267 entries, 0 to 2266
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   ticker        2267 non-null   object
 1   queryCount    2267 non-null   int64 
 2   resultsCount  2267 non-null   int64 
 3   adjusted      2267 non-null   bool  
 4   results       2267 non-null   object
 5   status        2267 non-null   object
 6   request_id    2267 non-null   object
 7   count         2267 non-null   int64 
dtypes: bool(1), int64(3), object(4)
memory usage: 126.3+ KB


# 3. Data Preprocessing (Variable Selection)

## Downloading & reading selected data

In [6]:
poly_data = json.loads(aggs.data)
df = pd.DataFrame(poly_data['results'])

with pd.option_context('display.max_rows', 10, 'display.max_columns', None,'display.max_colwidth', None):
    display(df)

,v,vw,o,c,h,l,t,n
0,199064828.0,24.9605,24.6625,25.0275,25.1088,24.6575,1411963200000,220213
1,221050556.0,25.2512,25.2025,25.1875,25.3850,25.1325,1412049600000,234940
2,205963876.0,24.8865,25.1475,24.7950,25.1725,24.6750,1412136000000,251176
3,191027712.0,24.8188,24.8175,24.9750,25.0550,24.5100,1412222400000,225041
4,173878340.0,24.9318,24.8600,24.9050,25.0525,24.7600,1412308800000,211827
...,...,...,...,...,...,...,...,...
2262,46172740.0,175.6665,174.2000,176.0800,176.9700,174.1500,1695614400000,564218
2263,64548945.0,172.7156,174.8200,171.9600,175.2000,171.6600,1695700800000,688340
2264,66920708.0,170.5335,172.6200,170.4300,173.0400,169.0500,1695787200000,707850
2265,56294419.0,170.2970,169.3400,170.6900,172.0300,167.6200,1695873600000,618584


## Understanding selected data

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2267 entries, 0 to 2266
Data columns (total 8 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   v       2267 non-null   float64
 1   vw      2267 non-null   float64
 2   o       2267 non-null   float64
 3   c       2267 non-null   float64
 4   h       2267 non-null   float64
 5   l       2267 non-null   float64
 6   t       2267 non-null   int64  
 7   n       2267 non-null   int64  
dtypes: float64(6), int64(2)
memory usage: 141.8 KB


# 4. Data Exploration 

## Retrieving selected stock data from AAPLE, Tesla, Meta, Google and Amazon.com

In [8]:
client = rest.RESTClient(config.API_KEY)

#setup dash
app = Dash(__name__)

app.layout = html.Div([
    html.H1('Stock Graphs  --select a stock asset from dropdown menu--'),
    dcc.Dropdown(id='stock-list',
    options = [
        {'label': 'Apple', 'value':'AAPL'},
        {'label': 'Tesla', 'value':'TSLA'},
        {'label': 'Meta', 'value':'META'},
        {'label': 'Google', 'value':'GOOG'},
        {'label': 'Amazon', 'value':'AMZN'}
    ],
    value = 'AAPL' # Apple as default value in dropdown menu
    ),
    dcc.Graph(id='Stock-graph'),
    ],
    style={'width':'500'}
    )


## A. Historical stock trends using interactive candlestick charts

In [9]:
#Callback decorator
@app.callback(Output('Stock-graph', 'figure'), [Input('stock-list', 'value')])

def graph(selected_dropdown_value):
    
    aggs = cast(
        HTTPResponse,
        client.get_aggs(
            selected_dropdown_value,
            1,
            'day',
            '2014-09-29',
            '2023-09-29',
            adjusted=True,
            raw=True,
            limit=50000
             
        ),
        
    )
    data = json.loads(aggs.data)
    
    for item in data:
        if item == 'results':
            rawData = data[item]
    
    closelist = []
    openlist = []
    highlist = []
    lowlist = []
    #volumelist = []#too much data for candlestick
    timelist = []
    
    for bar in rawData:
        for category in bar:
            if category == 'c':
                closelist.append(bar[category])
            elif category == 'o':
                openlist.append(bar[category])
            elif category == 'h':
                highlist.append(bar[category])
            elif category == 'l':
                lowlist.append(bar[category])
            elif category == 't':
                timelist.append(pd.Timestamp(bar[category], tz='GMT', unit='ms'))#to change GMT and milisecond formats
                                             
    fig = go.Figure()
    fig.add_trace(go.Candlestick(x=timelist, close=closelist, open=openlist, high=highlist, low=lowlist, name='Stock Data'))
    fig.update_layout(xaxis_rangeslider_visible=False, template='plotly_dark') #to eliminate slider plotly outputs by default and use dark mode 
    return fig


if __name__ == '__main__':
    app.run(debug=True, port=8050)

#Alternative access to charts by pasting this link, http://127.0.0.1:8050/ , on web browser

# B. Strategies for buying and selling stocks

### Select a stock asset for the calculations -- enter stock ticker at prompt, and then press the "enter" key on keyboard

In [10]:
varname = input().upper()
varname

 AAPL


'AAPL'

## Calculating the Exponential Moving Average (EMA) indicator

In [11]:
def getData():
     client = rest.RESTClient(config.API_KEY)
     closelist = []
     timelist = []
     #endDate = dt.datetime.today().date()#can be used for dynamic time/date updating (from 2022-08-23 onwards)
     
     aggs = cast(
        HTTPResponse,
        client.get_aggs(
        varname,
        1,
        'day',
        '2014-09-29',
        '2023-09-29',
        adjusted=True, 
        raw=True,
        limit=50000
            ),
    )
     data = json.loads(aggs.data)
     bars = data['results']
     
     for bar in bars:
         for value in bar:
             if value == 'c':
                 closelist.append(bar[value])
             elif value == 't':
                 timelist.append(pd.Timestamp(bar[value], tz='GMT', unit='ms'))

     #convert closelist into a numpy array for data processing using talib
     closelist = np.array(closelist)
     #get talib EMAs
     ema_eight = talib.EMA(closelist, 8)
     ema_thirteen = talib.EMA(closelist, 13)
     ema_twentyone = talib.EMA(closelist, 21)
     ema_fifty = talib.EMA(closelist, 55)
     
     print(f'\n\033[1mEMA_50 calculations for {varname} stocks:\n\033[0m')
     
     # checking for date when EMA_55 crosses above or under the other EMAs,
     # which marks the entry/buy and the exit/sell points, respectively.
     for i in range(len(ema_fifty)):
         if (ema_fifty[i] > ema_eight[i]) and (ema_fifty[i] > ema_thirteen[i]) and (ema_fifty[i] > ema_twentyone[i]):
             print("EMA_50 has crossed ABOVE ALL other EMAs on: " + str(timelist[i]) )
         elif(ema_fifty[i] < ema_eight[i]) and (ema_fifty[i] < ema_thirteen[i]) and (ema_fifty[i] < ema_twentyone[i]):
             print("EMA_50 has crossed UNDER ALL other EMAs on: " + str(timelist[i]) )
getData()


EMA_50 calculations for AAPL stocks:

EMA_50 has crossed UNDER ALL other EMAs on: 2014-12-15 05:00:00+00:00
EMA_50 has crossed UNDER ALL other EMAs on: 2014-12-16 05:00:00+00:00
EMA_50 has crossed UNDER ALL other EMAs on: 2014-12-17 05:00:00+00:00
EMA_50 has crossed UNDER ALL other EMAs on: 2014-12-18 05:00:00+00:00
EMA_50 has crossed UNDER ALL other EMAs on: 2014-12-19 05:00:00+00:00
EMA_50 has crossed UNDER ALL other EMAs on: 2014-12-22 05:00:00+00:00
EMA_50 has crossed UNDER ALL other EMAs on: 2014-12-23 05:00:00+00:00
EMA_50 has crossed UNDER ALL other EMAs on: 2014-12-24 05:00:00+00:00
EMA_50 has crossed UNDER ALL other EMAs on: 2014-12-26 05:00:00+00:00
EMA_50 has crossed UNDER ALL other EMAs on: 2014-12-29 05:00:00+00:00
EMA_50 has crossed UNDER ALL other EMAs on: 2014-12-30 05:00:00+00:00
EMA_50 has crossed UNDER ALL other EMAs on: 2014-12-31 05:00:00+00:00
EMA_50 has crossed UNDER ALL other EMAs on: 2015-01-02 05:00:00+00:00
EMA_50 has crossed UNDER ALL other EMAs on: 2015-01

## Calculating the Simple Moving Average (SMA) and the Relative Strength Index (RSI) indicators

### Select moving averages for SMA calculations
-- enter day-period at prompt, and then press the "enter" key on keyboard

In [16]:
n1 = int(input())
n2 = int(input())
print (f'Entered moving averages are: the {n1}-day and {n2}-day periods. -- value=70 and value=20, are the typically used values, respectively')

 70
 20


Entered moving averages are: the 70-day and 20-day periods. -- value=70 and value=20, are the typically used values, respectively


### Select upper, lower & RSI window values for RSI calculations -- enter values at prompt, and then press the "enter" key on keyboard

In [17]:
upper_bound = int(input())
lower_bound = int(input())
rsi_window = int(input())
print(f'Entered values: upper boundary is {upper_bound}, lower boundary is {lower_bound} and rsi moving window is {rsi_window}')
print('-- value=70, value=30, and value=14, are the typically used values, respectively')

 70
 30
 14


Entered values: upper boundary is 70, lower boundary is 30 and rsi moving window is 14
-- value=70, value=30, and value=14, are the typically used values, respectively


In [18]:
client = rest.RESTClient(config.API_KEY)

aggs = cast(
    HTTPResponse,
    client.get_aggs(
        varname,
        1,
        'day',
        '2014-09-29',
        '2023-09-29',
        adjusted=True,
        raw=True,
        limit=50000
    )
)

poly_data = json.loads(aggs.data)
poly_data = poly_data['results'] #to get the bars data only (e.g., o, h, l, etc.)

dates = []  
for bar in poly_data:
    for item in bar:
        if item == 't':
            dates.append(pd.Timestamp(bar[item], tz='GMT', unit='ms').date()) 

df = pd.DataFrame(poly_data)

d = {'Open' : df['o'], 'High' : df['h'], 'Low' : df['l'], 'Close': df['c'], 'Volume': df['v']} #get only bars needed for backtesting

dataFrame = pd.DataFrame(d)
pd.DateTimeIndex = dates #displays graph but no dates
#dataFrame.index = dates #first codeline tried but not working


#uncomment lines below to pirnt and compare df's
# print(df)
# print(dataFrame)

#Backtesting analysis SMA and RSI

class smacross(Strategy):
    def init(self):
        #n1 = 10
        #n2 = 20
        price = self.data.Close
        self.sma1 = self.I(SMA, price, n1)
        self.sma2 = self.I(SMA, price, n2)
    
    def next(self):
        if crossover(self.sma1, self.sma2):
            self.buy()
        elif crossover(self.sma2, self.sma1):
            self.sell()
bt = Backtest(dataFrame, smacross, commission =0.002, exclusive_orders=True) #exclusive_orders is an optional arg

print(f'\n\033[1mSMA Backtest Output on {varname} Stocks (Stats & Interactive Charts):\n\033[0m.')
print(bt.run())
print()
stats = bt.run()
bt.plot()



class rsioscillator(Strategy): #using typical values as described in talib and backtesting github page 
    upper_bound = upper_bound
    lower_bound = lower_bound
    rsi_window = rsi_window

    def init(self):
        self.rsi = self.I(talib.RSI, self.data.Close, self.rsi_window)

    def next(self):
        if crossover(self.rsi, self.upper_bound):
            self.position.close()
        elif crossover(self.lower_bound, self.rsi):
            self.buy()
bt = Backtest(dataFrame, rsioscillator, cash=10000, commission=.002)

print(f'\n\033[1mRSI Backtest Output on {varname} Stocks (Stats & Interactive Charts):\n\033[0m.')
print(bt.run())
print()
stats = bt.run()
bt.plot()


SMA Backtest Output on AAPL Stocks (Stats & Interactive Charts):
.
Start                                     0.0
End                                    2266.0
Duration                               2266.0
Exposure Time [%]                   96.382885
Equity Final [$]                  6320.874515
Equity Peak [$]                   13785.82435
Return [%]                         -36.791255
Buy & Hold Return [%]              584.087504
Return (Ann.) [%]                         0.0
Volatility (Ann.) [%]                     NaN
Sharpe Ratio                              NaN
Sortino Ratio                             NaN
Calmar Ratio                              0.0
Max. Drawdown [%]                  -82.863856
Avg. Drawdown [%]                  -17.510091
Max. Drawdown Duration                 1325.0
Avg. Drawdown Duration                  216.9
# Trades                                 39.0
Win Rate [%]                        61.538462
Best Trade [%]                      15.532034
Worst Trade 


RSI Backtest Output on AAPL Stocks (Stats & Interactive Charts):
.
Start                                     0.0
End                                    2266.0
Duration                               2266.0
Exposure Time [%]                   35.906484
Equity Final [$]                  24380.42987
Equity Peak [$]                   26873.30987
Return [%]                         143.804299
Buy & Hold Return [%]              584.087504
Return (Ann.) [%]                         0.0
Volatility (Ann.) [%]                     NaN
Sharpe Ratio                              NaN
Sortino Ratio                             NaN
Calmar Ratio                              0.0
Max. Drawdown [%]                  -29.233733
Avg. Drawdown [%]                   -4.992982
Max. Drawdown Duration                  650.0
Avg. Drawdown Duration              46.384615
# Trades                                 10.0
Win Rate [%]                             80.0
Best Trade [%]                       37.06073
Worst Trade 

GridPlot(id='p2249', ...)

# References
**A. &ensp;URL's with documentation of sortwares and libraries used for this project (url's last tested on 06/06/2024):**
1. &ensp;Stock Market API (version 1.2.0): https://polygon.io/
2. &ensp;Stocks API Documentation: https://polygon.io/docs/stocks/getting-started
3. &ensp;Polygon.IO API and Webpage Terms of Service: https://polygon.io/terms
4. &ensp;Backtesting.py (version 0.3.3): https://kernc.github.io/backtesting.py/doc/backtesting/#gsc.tab=0
5. &ensp;Technical Analysis Library (Ta-Lib, version 0.4.29): https://hexdocs.pm/talib/api-reference.html
6. &ensp;Dash (version 2.17.0): https://dash.plotly.com/
7. &ensp;Bokeh (version 3.2.1): https://docs.bokeh.org/en/latest/
8. &ensp;matplotlib (version 3.8.4): https://matplotlib.org/stable/users/index.html
9. &ensp;numpy (version 1.26.4): https://numpy.org/doc/
10. &ensp;plotly (version 5.22.0): https://plotly.github.io/plotly.py-docs/generated/plotly

**B. &ensp;URL's with tutorials relevant to the topics treated in this project (url's last tested on 06/06/2024):**
1. &ensp;EMA Strategy as way to Know When to Buy or Sell Stocks: https://www.youtube.com/watch?v=kGZh6X8HBw8\n
3. &ensp;How to Use the Relative Strength Index (RSI): https://www.youtube.com/watch?v=hbcCykbX14U
4. &ensp;EMA vs SMA - How to find the perfect Moving Average as a trader: https://www.youtube.com/watch?v=Z_gVO1FcXhs
5. &ensp;How To Use 20 SMA (Simple Moving Average): https://www.youtube.com/watch?v=WYiTLT9kwCQ

**C. &ensp;References cited in "Interpretation of Stock Trends and Strategy Results" (Section 5) in this notebook (url's last tested on 06/06/2024):**
1. Alan Northcott (2009). "The Complete Guide to Using Candlestick Charting: How to Earn High Rates of Return - Safely," Pages 15–17. Atlantic Publishing Group.
2. Cory Mitchell (2024). Understanding Basic Candlestick Charts. Investopedia, February 28, 2024. Url: https://www.investopedia.com/trading/candlestick-charting-what-is-it/
3. James Chen (2024). Backtesting: Definition, How It Works, and Downsides. Investopedia, April 11, 2024. Url: https://www.investopedia.com/terms/b/backtesting.asp
4. Tradeciety Group (2023). THE ULTIMATE GUIDE TO BACKTESTING. Tradeciety, December 5, 2023. Url: https://tradeciety.com/the-ultimate-guide-to-backtesting
5. J.B. Maverick (2024). How Is the Exponential Moving Average (EMA) Formula Calculated?, Investopedia, April 30, 2024. Url: https://www.investopedia.com/ask/answers/122314/what-exponential-moving-average-ema-formula-and-how-ema-calculated.asp#:~:text=The%20exponential%20moving%20average%20(EMA)%20is%20a%20technical%20chart%20indicator,importance%20to%20recent%20price%20data.
6. Niclas Hummel (2023). System Quality Number (SQN). Medium Blog, October 28, 2023. Url: https://medium.com/@niclas_hummel/system-quality-number-sqn-cb04cf7e9ecd#:~:text=The%20System%20Quality%20Number,indicates%20its%20stability%20or%20consistency.

**D. &ensp;References cited in "Learning & Conclusions" (Section 6) in this notebook (url's last tested on 06/06/2024):**
1. Saxo Group, Saxo Bank A/S. A guide to the 10 most popular trading indicators. Url: https://www.home.saxo/learn/guides/trading-strategies/a-guide-to-the-10-most-popular-trading-indicators
2. CoinDCX (2023). Top 10 Technical Indicators for Crypto Trading & Analysis. In CoinDCX Blog, November 9, 2023. Url: https://coindcx.com/blog/cryptocurrency/10-best-indicators-for-crypto-trading
3. Shobhit Seth (2024). The Top Technical Indicators for Options Trading. Investopedia, May 17, 2024. Url: https://www.investopedia.com/articles/active-trading/101314/top-technical-indicators-options-trading.asp
4. Robert Farrington (2023). 6 Weird But Successful Indicators For Stocks. The College Investor,June 19, 2023. Url: https://thecollegeinvestor.com/1666/weird-stock-market-indicators/
5. Rob Wile (2012). 15 Of The Most Spurious Correlations In The Stock Market. BusinessInsider Newsletter, April 19, 2012. Url: https://www.businessinsider.com/15-spurious-stock-correlations-2012-4
6. Jean Folger (2022). Backtesting and Forward Testing: The Importance of Correlation. Investopedia.com, July 12, 2022. Url: https://www.investopedia.com/articles/trading/10/backtesting-walkforward-important-correlation.asp